# Running An Armory Notebook [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/twosixlabs/armory/blob/master/notebooks/running_an_armory_notebook.ipynb)

In this tutorial, we'll demonstrate how to run Armory scenarios interactively, export and visualize data examples, and more. We'll use the CARLA object detection scenario as an example, although what follows is instructive for using Armory in general.


# Setup & Installation

Armory comes in a variety of formats- from a container, to a python package, and even a git repo- and can be installed in a number of configurations. 

## Google Colab
Below is an example *kickstart* script to get `armory` running in Google Colab.

In [ ]:
%%bash
# Only execute this cell if you are running this notebook in Google Colab

if [ ! -d armory ]; then
  mkdir -p armory-repo
  pushd armory-repo
    git clone https://github.com/twosixlabs/armory.git .
  popd
  cp -r armory-repo/* .
fi

pip install --upgrade pip
pip install armory-testbed[developer,engine,datasets,pytorch,tensorflow]

### Loading the Scenario
To load the scenario, we use Armory's `get()` function in `armory.scenarios.main`. The filepath used below is an example, and you'll receive the specific path logged to console after invoking `armory run`.

Calling `load()` will load the pieces (model, dataset, attack, etc.) necessary to run the scenario.

In [ ]:
import armory
from armory.scenarios.main import get as get_scenario

s = get_scenario(
      "./scenario_configs/eval5/carla_object_detection/carla_obj_det_dpatch_undefended.json"
    ).load()

2022-11-03 16:28:49 16m20s ERROR    armory.scenarios.main:_scenario_setup:41 eval_id not in config. Inserting current timestamp 1667492929.0279355
2022-11-03 16:28:49 16m20s INFO     armory.scenarios.main:_scenario_setup:49 armory outputs and logs will be written to /armory/outputs/1667492929.0279355
2022-11-03 16:28:52 16m24s INFO     armory.utils.external_repo:download_and_extract_repo:129 Downloading external repo: colour-science/colour@v0.3.16
2022-11-03 16:28:53 16m25s INFO     armory.data.utils:maybe_download_weights_from_s3:47 Using available carla_rgb_weights_eval5.pt in Armory `saved_model_dir`
2022-11-03 16:28:56 16m28s INFO     armory.scenarios.scenario:load_dataset:205 Loading test dataset carla_obj_det_dev...


### Running the Scenario

Now that we've instatiated the scenario object, we could simply call `s.evaluate()` to run the entire scenario from start to finish. However, in this tutorial, we'll demonstrate how to run the scenario in a step-by-step fashion.

To grab our first batch of data, we call the scenario's `next()` method. At this point, we can now access `x` and `y` as attributes of the scenario object, as well as `i` for the batch index:


In [ ]:
s.next()
s.x.shape

(1, 960, 1280, 3)


Armory scenarios, at a high level, execute in the following manner:

1. Load the next batch of data
2. Predict on the benign sample
3. Generate an adversarial sample and make a prediction on it

We did step 1 when we called `s.next()` earlier. To predict on the benign sample, we can call `s.run_benign()`. Similarly for step 3, we call `s.run_attack()`. Calling `s.evaluate_current()` would call both of these methods, but in this example we'll run each step one at a time:

In [ ]:
s.run_benign()

#### Prediction

The benign prediction is now accessible via the `y_pred` attribute:

In [ ]:
s.y_pred

# Attack

Next, we run the attack. For the sake of brevity in this tutorial, we'll modify the `"max_iter"` parameter of the attack first:

In [ ]:
s.config["attack"]["kwargs"]["max_iter"] = 5
s.load_attack()

s.run_attack()

RobustDPatch iteration:   0%|          | 0/5 [00:00<?, ?it/s]

At this point, we can access the adversarial sample via the `x_adv` attribute:

In [ ]:
s.x_adv.shape

(1, 960, 1280, 3)

To continue this process, we could keep calling `s.next()` followed by `s.run_benign()` and `s.run_attack()`. Once we'd like to close out the experiment, we could then call `s.finalize_results()` and observe the logged results. Instead, in the following section we'll show how to export and view data examples.

## Exporting and Viewing Data Examples

#### Exporting Data
Here we will show how to export data examples using the `sample_exporter` object. By "export", we mean write examples (.png, .wav, .mp4, etc. depending on the scenario) to the `~/.armory` experiment output directory. 

Each scenario object contains a `sample_exporter` which implements an `export()` method taking the following inputs:

```
def export(self, x, basename, **kwargs):
```

In the simplest case, to export the raw inputs without any annotation (e.g. bounding boxes), we can call the following (note that the `with_boxes` kwarg is specific to object detection scenarios):

The print statement was added to show the location of the Armory scenario output directory. If running in docker, note that this is the path inside the container and that the container path `/armory` should map to the host path `~/.armory`.

Now looking inside the scenario output directory, we see the image we just exported.

```
I have no name!@b24233d4dd79:/workspace$ ls /armory/outputs/2022-05-23T153707.378169/saved_samples
benign_x_batch_0.png

```

In [ ]:
batch_number = s.i
s.sample_exporter.export(s.x[0], f"benign_x_batch_{batch_number}", with_boxes=False)

print(s.sample_exporter.output_dir)

/armory/outputs/1667492929.0279355/saved_samples


This next part is specific to object detection, but to save the image with ground-truth and predicted boxes we can pass `y` and `y_pred` kwargs to the `export()` call:

In [ ]:
s.sample_exporter.export(
    s.x[0],
    f"benign_x_batch_{batch_number}_with_boxes",
    y=s.y[0],
    y_pred=s.y_pred[0],
    with_boxes=True,
)

#### Viewing Data

The `sample_exporter` class also makes it easy to retrieve viewable examples, specifically PIL objects in the case of image-related scenarios. These can be accessed via the `sample_exporter`'s `get_sample()` method which typically takes in a single input `x`. For scenarios with bounding boxes, this method also optionally takes in `y`, `y_pred`, and a boolean `with_boxes`. 

In [ ]:
x_pil = s.sample_exporter.get_sample(
    s.x[0], y=s.y[0], y_pred=s.y_pred[0], with_boxes=True
)
x_adv_pil = s.sample_exporter.get_sample(
    s.x_adv[0], y=s.y[0], y_pred=s.y_pred_adv[0], with_boxes=True
)

Now we can use Jupyter's built-in `display()` function to view the images:

In [ ]:
import io
import IPython.display as display
import ipywidgets as widgets

def image_to_byte_array(image):
  img_byte_arr = io.BytesIO()
  image.save(img_byte_arr, format='JPEG')
  return img_byte_arr.getvalue()


x_widget = widgets.Image(
      value=image_to_byte_array(x_pil),
      format='jpg',
      width=500,
      height=500
    )

x_adv_widget = widgets.Image(
      value=image_to_byte_array(x_adv_pil),
      format='jpg',
      width=500,
      height=500
    )

display.display(widgets.HBox([x_widget, x_adv_widget]))

## Finishing the Experiment

If you'd like to run the experiment over the rest of the data examples, simply call `s.evaluate()`. Doing so will complete the rest of the evaluation, prepare and finalize the results, and write the results JSON to the Armory scenario output directory.

If instead you'd like to finish the experiment at its current step, call `s.finalize_results()` which logs the experiment results to console.

In [ ]:
s.finalize_results()

2022-11-03 17:11:29 59m SUCCESS  armory.instrument.config:_write:225 benign_mean_categorical_accuracy on benign examples w.r.t. ground truth labels: 15.38%
2022-11-03 17:11:29 59m SUCCESS  armory.instrument.config:_write:225 adversarial_mean_categorical_accuracy on adversarial examples w.r.t. ground truth labels: 0.00%
2022-11-03 17:11:29 59m SUCCESS  armory.instrument.config:_write:225 benign_carla_od_AP_per_class on benign examples w.r.t. ground truth labels: {'mean': 0.89500000000000002, 'class': {1: 0.93999999999999995, 2: 0.84999999999999998}}
2022-11-03 17:11:29 59m SUCCESS  armory.instrument.config:_write:225 adversarial_carla_od_AP_per_class on adversarial examples w.r.t. ground truth labels: {'mean': 0.69999999999999996, 'class': {1: 0.69999999999999996, 2: 0.69999999999999996}}
2022-11-03 17:11:29 59m SUCCESS  armory.instrument.config:_write:225 benign_mean_carla_od_disappearance_rate on benign examples w.r.t. ground truth labels: 30.00%
2022-11-03 17:11:29 59m SUCCESS  armor

Afterwards, you can then call `s.save()` to write the results JSON to the Armory output directory. Alternatively, if you'd like to return the results as a Python dictionary, call `s.prepare_results()`.

In [ ]:
results_dict = s.prepare_results()
print(results_dict.keys())

dict_keys(['armory_version', 'config', 'results', 'timestamp'])


In [ ]:
print(results_dict["results"])

{'benign_mean_categorical_accuracy': [0.15384615384615385], 'adversarial_mean_categorical_accuracy': [0.0], 'perturbation_mean_linf': [0.9843137264251709], 'benign_carla_od_AP_per_class': [{'mean': 0.89500000000000002, 'class': {1: 0.93999999999999995, 2: 0.84999999999999998}}, {'mean': 0.89500000000000002, 'class': {1: 0.93999999999999995, 2: 0.84999999999999998}}], 'adversarial_carla_od_AP_per_class': [{'mean': 0.69999999999999996, 'class': {1: 0.69999999999999996, 2: 0.69999999999999996}}, {'mean': 0.69999999999999996, 'class': {1: 0.69999999999999996, 2: 0.69999999999999996}}], 'benign_mean_carla_od_disappearance_rate': [0.30000000000000004, 0.30000000000000004], 'adversarial_mean_carla_od_disappearance_rate': [0.30000000000000004, 0.30000000000000004], 'benign_mean_carla_od_hallucinations_per_image': [1.0, 1.0], 'adversarial_mean_carla_od_hallucinations_per_image': [2.0, 2.0], 'benign_mean_carla_od_misclassification_rate': [0.0, 0.0], 'adversarial_mean_carla_od_misclassification_r

In [ ]:
s.save()